<a href="https://colab.research.google.com/github/JSJeong-me/Retriever/blob/main/08-Contextual-Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://community.fullstackretrieval.com/document-transforms/contextual-compression

In [ ]:
!pip install openai
!pip install python-dotenv
!pip install langchain

In [2]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.6 MB/s eta 0:00:00


In [3]:
!echo "OPENAI_API_KEY=" >> .env
!source /content/.env

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
!pip install chromadb
!pip install tiktoken

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, model='gpt-4')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
# Loading a single website
loader = WebBaseLoader("http://www.paulgraham.com/superlinear.html")
docs = loader.load()

# Split your website into big chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7000, chunk_overlap=0)
chunks = text_splitter.split_documents(docs)

print (f"Your {len(docs)} documents have been split into {len(chunks)} chunks")


Your 1 documents have been split into 5 chunks


In [7]:
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vectordb.as_retriever(search_kwargs={"k": 2}))

In [10]:
relevant_docs = compression_retriever.base_retriever.get_relevant_documents("What do people think is a flaw of capitalism?")
print (f"You have {len(relevant_docs)} relevant docs")
print (relevant_docs[0].page_content[:500])
print (f"Your first document has length: {len(relevant_docs[0].page_content)}")

You have 2 relevant docs
gradual improvements in technique, not the discoveries of a few
exceptionally learned people.[3]
It's not mathematically correct to describe a step function as
superlinear, but a step function starting from zero works like a
superlinear function when it describes the reward curve for effort
by a rational actor. If it starts at zero then the part before the
step is below any linearly increasing return, and the part after
the step must be above the necessary return at that point or no one
would bo
Your first document has length: 3920


In [11]:
compressor.compress_documents(documents=relevant_docs, query="What do people think is a flaw of capitalism?")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[Document(page_content='"It\'s unclear exactly what advocates of "equity" mean by it. They seem to disagree among themselves. But whatever they mean is probably at odds with a world in which institutions have less power to control outcomes, and a handful of outliers do much better than everyone else.It may seem like bad luck for this concept that it arose at just the moment when the world was shifting in the opposite direction, but I don\'t think this was a coincidence. I think one reason it arose now is because its adherents feel threatened by rapidly increasing variation in performance."', metadata={'language': 'No language found.', 'source': 'http://www.paulgraham.com/superlinear.html', 'title': 'Superlinear Returns'}),
 Document(page_content="Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, p

In [12]:
question = "What do people think is a flaw of capitalism?"
compressed_docs = compression_retriever.get_relevant_documents(question)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [13]:
compressed_docs

[Document(page_content='"It\'s unclear exactly what advocates of "equity" mean by it.\nThey seem to disagree among themselves. But whatever they mean is\nprobably at odds with a world in which institutions have less power\nto control outcomes, and a handful of outliers do much better than\neveryone else.It may seem like bad luck for this concept that it arose at just\nthe moment when the world was shifting in the opposite direction,\nbut I don\'t think this was a coincidence. I think one reason it\narose now is because its adherents feel threatened by rapidly\nincreasing variation in performance."', metadata={'language': 'No language found.', 'source': 'http://www.paulgraham.com/superlinear.html', 'title': 'Superlinear Returns'}),
 Document(page_content="Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in

In [14]:
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [15]:
llm.predict(text=PROMPT.format_prompt(
    context=compressed_docs,
    question=question
).text)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Some people think that superlinear returns for performance, where a handful of outliers do much better than everyone else, is a flaw of capitalism.'